# Language

## number of products per Language

In [ ]:
WITH products AS (
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.software`
)
SELECT languageCode, COUNT( DISTINCT id) AS productCount
	FROM products
	GROUP BY languageCode, ORDER BY productCount DESC

## Language by country

In [ ]:
WITH products AS (
	SELECT id, type, languageCode FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, languageCode FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, languageCode FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, languageCode FROM `openaire-graph.oag_v10_6_0.software`
),
country_rel AS (
	SELECT r.source, r.target, o.countryCode
	FROM `oag_v10_6_0.relatiONs` r
	JOIN `oag_v10_6_0.organizatiONs` o ON r.target = o.id
		WHERE r.relatiONName = 'hASAuthorInstitutiON'
)
SELECT p.languageCode, r.countryCode, COUNT(DISTINCT p.id) AS productCount
	FROM products p
	JOIN country_rel r ON p.id = r.source
	
	GROUP BY languageCode, countryCode
	ORDER BY productCount DESC

# Formats

## Get Top-level research products formats

In [ ]:
WITH products AS (
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.software`
)
SELECT TYPE, COUNT(DISTINCT id) AS productCount
FROM `oag_v10_6_0.{table}`

GROUP BY type

## Get Fine-grained bibliotyping 

In [ ]:
WITH products AS (
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.software`
)
SELECT  JSON_VALUE(inst, "$.type") AS fineGrainedType, COUNT(DISTINCT id) AS productCount
FROM products,
	UNNEST(JSON_EXTRACT_ARRAY(instances)) AS inst

GROUP BY instType
ORDER BY productCount desc

## Format and Languages

In [ ]:
WITH products AS (
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.software`
),
country_rel AS (
	SELECT r.source, r.target, o.countryCode
	FROM `oag_v10_6_0.relatiONs` r
	JOIN `oag_v10_6_0.organizatiONs` o ON r.target = o.id
		WHERE r.relatiONName = 'hASAuthorInstitutiON'
)
SELECT JSON_VALUE(inst, "$.type") AS instType, p.languageCode, COUNT(DISTINCT p.id) AS productCount
	FROM `oag_v10_6_0.others` p,
		UNNEST(JSON_EXTRACT_ARRAY(p.instances)) AS inst
	JOIN country_rel r ON p.id = r.source
	
	GROUP BY instType, languageCode
	ORDER BY productCount desc

# Geographical RegiON

In [ ]:
WITH products AS (
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.software`
),
country_rel AS (
	SELECT r.source, r.target, o.countryCode
	FROM `oag_v10_6_0.relatiONs` r
	JOIN `oag_v10_6_0.organizatiONs` o ON r.target = o.id
		WHERE r.relatiONName = 'hASAuthorInstitutiON'
)
SELECT r.countryCode, count(distinct t.id) AS productCount
FROM `oag_v10_6_0.{table}` t
JOIN country_rel r ON p.id = r.source

GROUP BY countryCode
ORDER BY productCount desc

## Countries total number of products

In [ ]:
WITH products AS (
	SELECT id FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id FROM `openaire-graph.oag_v10_6_0.software`
)
SELECT o.countryCode, count(distinct p.id)
FROM `oag_v10_6_0.relatiONs` r
JOIN products p ON r.source = p.id
JOIN `oag_v10_6_0.organizatiONs` o ON r.target = o.id

	WHERE r.relatiONName = 'hASAuthorInstitutiON'

GROUP BY o.countryCode

# InstitutiON

## Number of InstitutiONs per Country

In [ ]:
SELECT countryCode, COUNT(DISTINCT ID) AS organisatiONCount
FROM `oag_v10_6_0.organizatiONs`
	WHERE countryCode IS NOT NULL

GROUP BY countryCode
ORDER BY organisatiONCount DESC

## OrganizatiONs Top-Level Research Type DistributiON

In [ ]:
WITH organizatiONs AS (
	SELECT o.id, org.organizatiONtype
	FROM `oag_v10_6_0.organizatiONs` o
	JOIN `cnr_internal.organizatiON-types` org
	ON o.id = org.id
),
products AS (
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type FROM `openaire-graph.oag_v10_6_0.software`
)
SELECT o.organizatiONtype, p.type, count(distinct p.id)
FROM `oag_v10_6_0.relatiONs` r
JOIN products p ON r.source = p.id
JOIN organizatiONs o ON r.target = o.id
	WHERE r.relatiONName = 'hASAuthorInstitutiON'

GROUP BY o.organizatiONtype, p.type


# Subjects

## Fields of Science (FoS)

In [ ]:
WITH products AS (
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.software`
)
SELECT JSON_VALUE(subject.subject.value) AS fos, COUNT(DISTINCT id) AS nProducts
FROM products,
	UNNEST(JSON_EXTRACT_ARRAY(subjects)) AS subject
	WHERE JSON_VALUE(subject.subject.scheme) = "FOS"

GROUP BY fos

## FoS distributiON per Country

In [ ]:
WITH products AS (
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.software`
),
country_rel AS (
	SELECT r.source, r.target, o.countryCode
	FROM `oag_v10_6_0.relatiONs` r
	JOIN `oag_v10_6_0.organizatiONs` o ON r.target = o.id
		WHERE r.relatiONName = 'hASAuthorInstitutiON'
)
SELECT JSON_VALUE(subject.subject.value) AS fos, r.countryCode, count(distinct id) AS nProducts
FROM products p,
	UNNEST(JSON_EXTRACT_ARRAY(subjects)) AS subject
JOIN country_rel r ON p.id = r.source
	WHERE JSON_VALUE(subject.subject.scheme) = "FOS"

GROUP BY fos, r.countryCode

## Substainable Development Goals (SDG)

In [ ]:
WITH products AS (
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.software`
)
SELECT JSON_VALUE(subject.subject.value) AS SDG, count(distinct id) AS nProducts
FROM products,
	UNNEST(JSON_EXTRACT_ARRAY(subjects)) AS subject

	WHERE JSON_VALUE(subject.subject.scheme) = "SDG"
GROUP BY SDG

## SDG DistributiON per Country

In [ ]:
WITH products AS (
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, subjects FROM `openaire-graph.oag_v10_6_0.software`
),
country_rel AS (
	SELECT r.source, r.target, o.countryCode
	FROM `oag_v10_6_0.relatiONs` r
	JOIN `oag_v10_6_0.organizatiONs` o ON r.target = o.id
	WHERE r.relatiONName = 'hASAuthorInstitutiON'
)
SELECT JSON_VALUE(subject.subject.value) AS SDG, r.countryCode, count(distinct id) AS nProducts
FROM products p,
UNNEST(JSON_EXTRACT_ARRAY(subjects)) AS subject
JOIN country_rel r ON p.id = r.source
WHERE JSON_VALUE(subject.subject.scheme) = "SDG"
GROUP BY SDG, r.countryCode

# DatASources

## Type of datASources

In [ ]:
SELECT distinct type, count(distinct id) AS datASourcesCount
FROM `oag_v10_6_0.datASources`
GROUP BY type

## Country of Publishers

In [ ]:
with filtered_relatiONs AS (
SELECT r.source, r.relatiONName, r.target
FROM `oag_v10_6_0.datASources` d
JOIN `oag_v10_6_0.relatiONs` r ON d.id = r.source
WHERE r.relatiONName = 'isProvidedBy' and r.targetType = 'organizatiON' and d.type = "Journal"
)
SELECT o.countryCode, count(distinct t.source) AS nJournals
FROM test t
JOIN `oag_v10_6_0.organizatiONs` o ON t.target = o.id
GROUP BY o.countryCode

# Funders

## Distinct Funders

In [ ]:
SELECT JSON_VALUE(funding, '$.shortName') AS fundingId, count(distinct id) AS nProjects
FROM `oag_v10_6_0.projects`,
	UNNEST(JSON_QUERY_ARRAY(fundings)) AS funding
GROUP BY fundingId

## Funders & Multilinguism

In [ ]:
WITH products AS (
	SELECT id, type, languageCode FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, languageCode FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, languageCode FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, languageCode FROM `openaire-graph.oag_v10_6_0.software`
),
enriched_products AS (
	SELECT r.source AS projectId, r.target AS productId, r.relatiONName, pr.fundings, p.languageCode
	FROM oag_v10_6_0.relatiONs r
	JOIN `oag_v10_6_0.projects` pr ON r.source = pr.id
	JOIN products p ON r.target = p.id
		WHERE r.relatiONName = 'produces'
)
SELECT JSON_VALUE(funding.shortName) AS funderId, JSON_VALUE(funding.name) AS funderName, count(distinct languageCode) AS languageCount, count(distinct productId) AS productsCount
FROM `enriched_products`,
	UNNEST(JSON_QUERY_ARRAY(fundings)) AS funding
GROUP BY funderId, funderName


## Funders and top level research product type 

In [ ]:
WITH products AS (
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.software`
),
enriched_products AS (
	SELECT r.source AS projectId, r.target AS productId, r.relatiONName, pr.fundings, p.instances
	FROM oag_v10_6_0.relatiONs r
	JOIN `oag_v10_6_0.projects` pr ON r.source = pr.id
	JOIN products p ON r.target = p.id
		WHERE r.relatiONName = 'produces'
)
SELECT JSON_VALUE(funder.shortName) AS funderId, JSON_VALUE(funder.name) AS funderName, JSON_VALUE(inst.type) AS productType, count(distinct productId) AS nProduct
FROM enriched_products ,
	UNNEST(JSON_EXTRACT_ARRAY(fundings)) AS funder,
	UNNEST(JSON_EXTRACT_ARRAY(instances)) AS inst

GROUP BY funderId, funderName, productType
ORDER BY funderName, productType ASc

## Funders and Sustainable Developmente Goals

In [ ]:
WITH products AS (
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.publicatiONs`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.others`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.datASets`
	UNION ALL
	SELECT id, type, instances FROM `openaire-graph.oag_v10_6_0.software`
),
enriched_products AS (
	SELECT r.source AS projectId, r.target AS productId, r.relatiONName, pr.fundings, p.subjects, p.type
	FROM oag_v10_6_0.relatiONs r
	JOIN `oag_v10_6_0.projects` pr ON r.source = pr.id
	JOIN products p ON r.target = p.id
		WHERE r.relatiONName = 'produces'
)
SELECT JSON_VALUE(funding, "$.shortName") AS funderId, JSON_VALUE(funding, "$.name") AS funderName, JSON_VALUE(subject.subject.value) AS SDG, count(distinct productId) AS n_products
FROM `temp_product`,
	UNNEST(JSON_QUERY_ARRAY(fundings)) AS funding,
	UNNEST(JSON_QUERY_ARRAY(subjects)) AS subject
	
	WHERE JSON_VALUE(subject.subject.scheme) = "SDG"
GROUP BY funderId, funderName, SDG